# 🚀 Web-Scraping als Methode der Korpuserstellung

<div class="alert alert-block alert-info"> <b> 🔔 Feinlernziel(e) dieses Kapitels</b></br>
Sie können verschiedene Methoden der Website-Abfrage aufzählen und Unterschiede identifizieren. </br>
Sie können Vor- und Nachteile der Methoden erklären und ermitteln, in welchen Szenarien welche Methode geeignet ist.
</div>

## Hinweise zur Ausführung des Notebooks
Dieses Notebook kann auf unterschiedlichen Levels erarbeitet werden (siehe Abschnitt ["Technische Voraussetzungen"](../introduction/introduction_requirements)): 
1. Book-Only Mode
2. Cloud Mode: Dafür auf 🚀 klicken und z.B. in Colab ausführen.
3. Local Mode: Dafür auf Herunterladen ↓ klicken und ".ipynb" wählen. 

## Hinweise zur Ausführung des Notebooks
Dieses Notebook kann auf unterschiedlichen Levels erarbeitet werden (siehe Abschnitt ["Technische Voraussetzungen"](../markdown/introduction_requirements)): 
1. Book-Only Mode
2. Cloud Mode: Dafür auf 🚀 klicken und z.B. in Colab ausführen.
3. Local Mode: Dafür auf Herunterladen ↓ klicken und ".ipynb" wählen. 

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
  
<b>Voraussetzungen zur Ausführung des Jupyter Notebooks:</b>
<ul>
<li> Installieren der Bibliotheken </li>
</ul>
Zum Testen: Ausführen der Zelle "load libraries".</br>
Alle Zellen, die mit 🚀 gekennzeichnet sind, werden nur bei der Ausführung des Noteboos in Colab / JupyterHub bzw. lokal ausgeführt. 
</details>

In [ ]:
#  🚀 Install libraries 
! pip install requests beautifulsoup4 scrapy selenium webdriver_manager time

## Einführung
Im [vorherigen Kapitel](scraping-intro_http-intro) haben wir bereits ein Beispiel zur automatisierten Abfrage einer Website kennengelernt. Um mehr als eine Website abzufragen, gibt es verschiedene Methoden. Welche Methode sich am besten zur Extraktion eignet, hängt davon ab, wie die abzufragenden Websites aufgebaut sind und ob sie rein statische oder auch dynamische Inhalte beinhalten. 

### Statische vs. dynamische Websites

Websites können grundsätzlich in zwei Kategorien eingeteilt werden: statische und dynamische Websites. Abhängig davon, welche Inhalte extrahiert werden sollen und wie die Website beschaffen ist, muss die Scraping-Methode angepasst werden.

- **Statische Websites**: Diese Websites sind fertige Dokumente, die auf einem Server bereitliegen. Wenn Sie eine solche Website anfordern, wird Ihnen exakt dieser vorbereitete Inhalt geschickt. Das ist vergleichbar mit dem Anfordern eines bestimmten Buches aus einer Bibliothek – der Inhalt liegt fertig vor und ändert sich nicht. Diese Art von Websites kann leicht mit einfachen Scraping-Methoden extrahiert werden, da alle Informationen direkt im HTML-Code enthalten sind.

- **Dynamische Websites**: Diese Websites werden erst im Moment der Anfrage zusammengestellt. Sie enthalten oft JavaScript-Code, der nach dem Laden der Seite ausgeführt wird und weitere Inhalte nachladen oder verändern kann. Das ist vergleichbar mit einem Koch, der das Gericht erst auf Bestellung zubereitet. Für die automatisierte Abfrage dieser Art von Websites benötigt man fortgeschrittenere Scraping-Methoden wie Selenium, die einen Browser simulieren können.

## Drei Ebenen des Web Scrapings

### 1. Einfache Anfragen mit requests

Die grundlegendste Form des Web Scrapings ist das Abrufen einzelner Webseiten, z.B. mit Hilfe des Python-Pakets `requests`. Diese Methode eignet sich für statische Webseiten, deren Inhalt direkt im HTML-Code enthalten ist.

In [6]:
# import library to perform HTTP requests
import requests

# Set URL 
url = "https://www.berlin.de/rbmskzl/"

# perform get request
response = requests.get(url)

# check if request was successful (code: 200)
if response.status_code == 200:
    print(f"Status-Code: {response.status_code}")

    # display the first lines of the response body (the content of the website)
    print("\nBeginn des HTML-Dokuments:")
    print(response.text[:100])
else:
    print(f"Fehler beim Abrufen der Seite: {response.status_code}")

Status-Code: 200

Beginn des HTML-Dokuments:
<!doctype html>
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="de" lang="de" data-lang="de">
<


**Vorteile:**
- Einfach zu implementieren
- Geringer Ressourcenverbrauch
- Ausreichend für einfache Scraping-Aufgaben

**Nachteile:**
- Nur einzelne Seiten werden abgerufen
- Keine automatische Navigation zu anderen Seiten
- Nicht geeignet für dynamisch generierte Inhalte (JavaScript)

### 2. Navigation mit Scrapy

Für komplexere Scraping-Aufgaben, bei denen mehrere Seiten durchlaufen werden müssen, eignet sich die Bibliothek `scrapy`. Sie ermöglicht das systematische Folgen von Links und das Extrahieren von Daten aus mehreren Seiten.

In [12]:
import scrapy
from scrapy.crawler import CrawlerProcess
import json

class SenatSpider(scrapy.Spider):
    name = 'senat'
    start_urls = ['https://www.berlin.de/rbmskzl/']

    def parse(self, response):
        # Extrahiere Daten von der aktuellen Seite
        for book in response.css('div'):
            yield {
                'title': book.css('h2.title::text').get(),
            }

        # Folge den Links zu den nächsten Seiten
        next_page = response.css('a.next-page::attr(href)').get()
        if next_page is not None:
            yield response.follow(next_page, self.parse)

# Run the spider in Jupyter
process = CrawlerProcess({
    'USER_AGENT': 'mozilla/4.0 (compatible; msie 7.0; windows nt 5.1; gtb5)',
    'FEEDS': {
        'output.json': {'format': 'json'},
    }
})

process.crawl(SenatSpider)
process.start()  # This will block until crawling is finished

2025-06-05 16:58:21 [scrapy.utils.log] INFO: Scrapy 2.13.1 started (bot: scrapybot)
2025-06-05 16:58:21 [scrapy.utils.log] INFO: Versions:
{'lxml': '5.4.0',
 'libxml2': '2.13.8',
 'cssselect': '1.3.0',
 'parsel': '1.10.0',
 'w3lib': '2.3.1',
 'Twisted': '24.11.0',
 'Python': '3.12.8 (main, Dec  6 2024, 19:42:06) [Clang 18.1.8 ]',
 'pyOpenSSL': '25.1.0 (OpenSSL 3.5.0 8 Apr 2025)',
 'cryptography': '45.0.3',
 'Platform': 'macOS-15.5-arm64-arm-64bit'}
2025-06-05 16:58:21 [scrapy.addons] INFO: Enabled addons:
[]
2025-06-05 16:58:21 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-06-05 16:58:21 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-06-05 16:58:21 [scrapy.extensions.telnet] INFO: Telnet Password: de83549a9333e40b
2025-06-05 16:58:21 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensio

RuntimeError: This event loop is already running

2025-06-05 16:58:21 [scrapy.core.engine] INFO: Spider opened
2025-06-05 16:58:21 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2025-06-05 16:58:21 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2025-06-05 16:58:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.berlin.de/rbmskzl/> (referer: None)
2025-06-05 16:58:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.berlin.de/rbmskzl/>
{'title': None}
2025-06-05 16:58:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.berlin.de/rbmskzl/>
{'title': 'Olympia-Bewerbung'}
2025-06-05 16:58:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.berlin.de/rbmskzl/>
{'title': None}
2025-06-05 16:58:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.berlin.de/rbmskzl/>
{'title': None}
2025-06-05 16:58:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.berlin.de/rbmskzl/>
{'title': None}
2025-06-05 16:58

**Vorteile:**
- Effizientes Crawlen mehrerer Seiten
- Integrierte Funktionen für Datenverwaltung und -export
- Robuste Error-Handling-Mechanismen

**Nachteile:**
- Steilere Lernkurve als bei `requests`
- Nicht geeignet für dynamische Webseiten mit JavaScript
- Komplexere Konfiguration

### 3. Simulation von Benutzerinteraktionen mit Selenium

Für Websites, die dynamische Inhalte mittels JavaScript laden oder Benutzerinteraktionen erfordern, ist `Selenium` die geeignete Wahl. Diese Bibliothek steuert einen echten Webbrowser und kann somit mit allen Elementen interagieren.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Browser-Instanz erstellen
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Website aufrufen
driver.get('https://example.com/dynamic-page')

# Warten, bis JavaScript-Inhalte geladen sind
time.sleep(2)

# Mit Elementen interagieren
search_button = driver.find_element(By.ID, 'search-button')
search_button.click()

# Auf dynamisch geladene Inhalte zugreifen
results = driver.find_elements(By.CLASS_NAME, 'result-item')
for result in results:
    print(result.text)

# Browser schließen
driver.quit()

**Vorteile:**
- Zugriff auf dynamisch geladene Inhalte (JavaScript)
- Simulation von Benutzerinteraktionen (Klicks, Formulare ausfüllen)
- "Sieht" die Website wie ein menschlicher Benutzer

**Nachteile:**
- Deutlich ressourcenintensiver
- Langsamer als `requests` oder `Scrapy`
- Anfälliger für Änderungen im Website-Layout

## Geeignete Szenarien für die verschiedenen Methoden

| Szenario | Geeignete Methode | Begründung |
|----------|-------------------|------------|
| Extraktion von Texten aus einer bekannten Webseite | `requests` | Einfach, effizient für einzelne statische Seiten |
| Durchsuchen und Extraktion von Daten aus einem Blog oder Wiki | `Scrapy` | Effizientes Folgen von Links, Extrahieren ähnlicher Daten von mehreren Seiten |
| Daten aus einem Social-Media-Portal | `Selenium` | Notwendig für Login, Scrollen, Klicken und dynamisch nachgeladene Inhalte |
| Korpuserstellung aus statischen Webseiten | `Scrapy` | Gute Balance aus Geschwindigkeit und Funktionalität für größere Sammlungen |
| Korpuserstellung aus dynamischen Webseiten | `Selenium` | Notwendig für Scrollen, Klicken und dynamisch nachgeladene Inhalte |
| Interaktion mit Suchformularen | `Selenium` | Ermöglicht das Ausfüllen und Absenden von Formularen |

## Ethische und rechtliche Aspekte

Beim Web Scraping sind stets ethische und rechtliche Aspekte zu beachten:

- Beachtung der `robots.txt`-Datei einer Website, die Informationen darüber gibt, welche Websites gescraped werden dürfen.
- Angemessene Wartezeiten zwischen Anfragen einhalten
- Keine persönlichen Daten ohne Einwilligung sammeln
- Urheberrecht und Nutzungsbedingungen der Websites beachten
- Datenschutzbestimmungen einhalten